In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# compress the trained model
import pickle
SAVED_MODEL_PATH = pathlib.Path('/home/jovyan/work/upload/data/trained_models')

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics, log_artifact
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'


# Math stuff
import numpy as np

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC
# :gamma: is a parameter for non linear hyperplanes. The higher the gamma
#         value it tries to exactly fit the training data set.
GAMMA = 3
# :C: is the penalty parameter of the error term. It controls the trade
#     off between smooth decision boundary and classifying the training points correctly.
#     !!! Increasing C values may lead to overfitting the training data. !!!
C = 2

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True

# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Subcategory']

## law2vec model

In [3]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


In [4]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    ("SVM", SVC(gamma=GAMMA, C=C))])

## Train SVM Model

In [5]:
# split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [6]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7f5072c865e0>),
                ('SVM', SVC(C=2, gamma=3))])

### Save the model into pickle file

In [9]:
with open(SAVED_MODEL_PATH / 'subcategory_svm_law2vec.pkl', 'wb') as pickle_model:
    pickle.dump(svm_l2v, pickle_model)

In [10]:
# Model Evaluation
svm_l2v.score(X_train, y_train)

0.9862385321100917

In [11]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([14, 11,  0,  2, 20,  9, 10, 10, 10, 12, 10, 22, 10,  0,  0, 21, 12,
       33, 21,  9, 14, 10,  0, 10, 12,  0, 10,  2, 12, 20, 24, 14, 14, 10,
        0, 20, 21, 16,  2,  9,  4, 24, 26, 21, 14, 29, 30, 20,  9, 30, 26,
       20, 14, 26, 28, 14,  9, 14, 12,  9, 14, 10,  0,  2,  9,  0, 16, 14,
       20, 10,  2, 10,  0,  0, 27, 10, 12, 14, 15, 21, 24,  4,  0, 10, 10,
        0,  0, 21, 21, 10, 10,  0,  0, 14, 27, 13,  9, 10,  0, 10, 10, 10,
       14, 21, 14, 10, 10, 20,  9, 21,  2,  0,  9,  0, 22, 14,  2, 20,  9,
       10,  9, 14,  9, 12,  2, 26, 14, 26, 10, 20, 10,  9,  9,  9, 10, 21,
       10, 10,  2,  9, 19, 21,  4, 20, 21, 16, 10, 10,  2, 30,  0, 14, 18,
       14, 22,  2, 10,  0,  4, 10,  0, 21,  4, 14, 22, 10, 30, 10, 12,  9,
        2, 27, 30, 22, 20, 10, 10, 10, 13,  0, 26,  4, 14, 10,  0,  0,  0,
       10, 13, 10, 10, 21, 15, 10, 19, 30, 22,  0, 18, 30, 21, 22,  0, 14,
       10, 14,  4, 12,  0, 19, 14, 10, 10, 30,  0,  9,  2,  9, 10,  0, 10,
       16, 20, 27, 30, 10

## Evaluation

In [12]:
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.555160
1,Precision,0.416806
2,Recall,0.376620
3,F1-Score,0.365535
4,Mean Absolute Error,4.896797
5,Mean Squared Error,82.804270


In [13]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation

{'Accuracy': 0.5551601423487544,
 'Precision': 0.4168059400034322,
 'Recall': 0.37662046146894634,
 'F1-Score': 0.36553472083749117,
 'Mean Absolute Error': 4.896797153024911,
 'Mean Squared Error': 82.80427046263345}

In [14]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Subategory (SVM, Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "Gamma": GAMMA,
                  "C": C}
    log_params(parameters)

    log_metrics(transformation)

    log_artifact('/home/jovyan/work/upload/data/trained_models/subcategory_svm_law2vec.pkl')